# 1. Parameters

In [1]:
# Defaults
cases_dir = 'cases/unset'
reference_file = 'references/NC_045512.gbk.gz'
input_files_all = 'input/input-files.tsv'
iterations = 3
mincov = 10
ncores = 32
number_samples = 10
build_tree = False

In [2]:
# Parameters
cases_dir = "cases/case-500"
iterations = 3
number_samples = 500
build_tree = True


In [3]:
from pathlib import Path
from shutil import rmtree
from os import makedirs
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

if cases_dir_path.exists():
    rmtree(cases_dir_path)
    
if not cases_dir_path.exists():
    makedirs(cases_dir_path)

input_files_all = Path(input_files_all)
reference_file = Path(reference_file)

case_name = str(cases_dir_path.name)
reference_name = reference_file.name.split('.')[0]

cases_input = cases_dir_path / 'input-files-case.tsv'
index_path = cases_dir_path / 'index'
benchmark_path = cases_dir_path / 'index-info.tsv'
output_tree = cases_dir_path / 'tree.tre'

# 2. Create subset input

In [4]:
import pandas as pd

all_input_df = pd.read_csv(input_files_all, sep='\t')
all_input_total = len(all_input_df)

subset_input_df = all_input_df.head(number_samples)
subset_input_total = len(subset_input_df)

subset_input_df.to_csv(cases_input, sep='\t', index=False)

print(f'Wrote {subset_input_total}/{all_input_total} samples to {cases_input}')

Wrote 500/100000 samples to cases/case-500/input-files-case.tsv


# 2. Index genomes

In [5]:
!gdi --version

gdi, version 0.3.0.dev12


## 2.1. Index reads

In [6]:
results_handler = gdi_benchmark.BenchmarkResultsHandler(name=case_name)
benchmarker = gdi_benchmark.IndexBenchmarker(benchmark_results_handler=results_handler,
                                             index_path=index_path, input_files_file=cases_input,
                                             reference_file=reference_file, mincov=mincov,
                                             build_tree=build_tree,
                                             ncores=ncores)

benchmark_df = benchmarker.benchmark(iterations=iterations)


Iteration 1 of index/analysis of 500 samples with 32 cores
Removing any extra snakemake directories: ['snakemake-assemblies.1631208292.8445542']


Creating new index: [gdi init cases/case-500/index]


Creating a new index took 2.80 seconds
Analysis running: [gdi --project-dir cases/case-500/index --ncores 32 analysis --use-conda --no-load-data --reference-file references/NC_045512.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/case-500/input-files-case.tsv]


Analysis took 4.59 minutes
Index running: [gdi --project-dir cases/case-500/index --ncores 32 load vcf-kmer --reference-file references/NC_045512.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/sars-cov-2/snakemake-assemblies.1631212461.7115388/gdi-input.fofn]


Indexing took 0.62 minutes
Building tree: [gdi --project-dir cases/case-500/index --ncores 32 rebuild tree --align-type full --extra-params '--fast -m GTR+F+R4' NC_045512]


...

Analysis took 4.80 minutes
Index running: [gdi --project-dir cases/case-500/index --ncores 32 load vcf-kmer --reference-file references/NC_045512.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/sars-cov-2/snakemake-assemblies.1631252814.328321/gdi-input.fofn]


Indexing took 0.61 minutes
Building tree: [gdi --project-dir cases/case-500/index --ncores 32 rebuild tree --align-type full --extra-params '--fast -m GTR+F+R4' NC_045512]


Building tree took 323.54 minutes


Attempting to set global database_path_translator=<genomics_data_index.storage.model.db.DatabasePathTranslator.DatabasePathTranslator object at 0x7f9547d76d90> but it is already set


A reminder to myself to look for a Python solution for directory sizes (instead of running `du`)


In [7]:
benchmark_df

,Name,Iteration,Number samples,Number features (all),Number features (no unknown),Number cores,Reference length,Analysis runtime,Analysis memory (max),Analysis memory (max/process),Analysis disk uage,Index runtime,Index memory (max),Index memory (max/process),Index size,Tree runtime,Tree memory (max),Tree memory (max/process),Total runtime,Max memory
0,case-500,1,500,30529,1396,32,29903,275.42,1.327899e+09,379527168.0,3.541881e+09,36.97,6.117863e+09,638693376.0,13262848.0,21123.0,888365056.0,650457088.0,21435.39,6.117863e+09
0,case-500,2,500,30529,1396,32,29903,284.68,1.253143e+09,371269632.0,3.541860e+09,37.00,6.019080e+09,626896896.0,13258752.0,18540.0,887410688.0,650592256.0,18861.68,6.019080e+09
0,case-500,3,500,30529,1396,32,29903,287.99,1.366954e+09,374501376.0,3.542491e+09,36.73,6.003589e+09,638357504.0,13262848.0,19383.0,882360320.0,644943872.0,19707.72,6.003589e+09


In [8]:
benchmark_df.to_csv(benchmark_path, sep='\t', index=False)

# 3. Export trees

In [9]:
if build_tree:
    !gdi --project-dir {index_path} export tree {reference_name} > {output_tree}
    print(f'Wrote tree to {output_tree}')
else:
    print(f'build_tree={build_tree} so no tree to export')

Wrote tree to cases/case-500/tree.tre
